In [1]:
import numpy as np
import pandas as pd
import hprotein
from matplotlib import pyplot as plt
import random
import cv2


/Users/farrar/py3.6.5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
# constants
LABEL_PATH = '../stage1_labels/train.csv'
TRAIN_PATH = '../stage1_train'

In [4]:
df = pd.read_csv(LABEL_PATH)
df.head()
lookup = df.values

In [5]:
specimen_ids = df['Id'].values
labels = df['Target'].values

print(specimen_ids.shape)
print(specimen_ids[0])
print(specimen_ids.shape)
print(labels[0:4])

(31072,)
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0
(31072,)
['16 0' '7 1 2 0' '5' '1']


In [6]:
result_df = pd.DataFrame()

In [7]:
split_labels = []

# split the space separated multi-label into a list of individual labels
split_label = labels[0].split(' ')
print (split_label)

# set up a numpy array to receive the encoded label
l = np.zeros(28)

# turn on the positive columns in the labels array
for label in split_label:
    l[np.uint8(label)] = 1

# append to the split labels array
split_labels.append(l)

split_labels = np.array(split_labels)
print(split_labels.shape)
print(split_labels[0])


['16', '0']
(1, 28)
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [8]:
split_labels = []
# loop through each specimen_id
for i in range(specimen_ids.shape[0]):
    
    # split the space separated multi-label into a list of individual labels
    split_label = labels[i].split(' ')

    # set up a numpy array to receive the encoded label
    l = np.zeros(28)

    # turn on the positive columns in the labels array
    for label in split_label:
        l[np.uint8(label)] = 1

    # append to the split labels array
    split_labels.append(l)
    
split_labels = np.array(split_labels)
print(split_labels.shape)
print(split_labels[0:4])

(31072, 28)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [9]:
from sklearn.cluster import KMeans

n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters).fit(split_labels)
print(kmeans.labels_)
cluster = kmeans.predict(split_labels)
print (cluster)

[2 1 4 ... 6 3 1]
[2 1 4 ... 6 3 1]


In [10]:
print (cluster.shape)
    


(31072,)


In [11]:
for i in range(n_clusters):
    print('Count of cluster {} -> {}'.format(i, (cluster==i).sum()))


Count of cluster 0 -> 7656
Count of cluster 1 -> 2580
Count of cluster 2 -> 7848
Count of cluster 3 -> 4374
Count of cluster 4 -> 1307
Count of cluster 5 -> 2002
Count of cluster 6 -> 1674
Count of cluster 7 -> 3631


In [12]:
data_set = []

for i in range(specimen_ids.shape[0]):
    data_set.append([specimen_ids[i], cluster[i]])
    
data_set = np.asarray(data_set)
print(data_set.shape)
print(data_set[0:4, :])

(31072, 2)
[['00070df0-bbc3-11e8-b2bc-ac1f6b6435d0' '2']
 ['000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0' '1']
 ['000a9596-bbc4-11e8-b2bc-ac1f6b6435d0' '4']
 ['000c99ba-bba4-11e8-b2b9-ac1f6b6435d0' '0']]


In [13]:
rods_rings = ['e403806e-bbbf-11e8-b2bb-ac1f6b6435d0','05d32f36-bba3-11e8-b2b9-ac1f6b6435d0',
              '0afda11a-bba0-11e8-b2b9-ac1f6b6435d0','18df69fc-bbb5-11e8-b2ba-ac1f6b6435d0',
              '2b3ce424-bba8-11e8-b2ba-ac1f6b6435d0','43f6bd88-bbc5-11e8-b2bc-ac1f6b6435d0',
              '70b97ed2-bbac-11e8-b2ba-ac1f6b6435d0','802998d4-bbbb-11e8-b2ba-ac1f6b6435d0',
              'b1131086-bb9f-11e8-b2b9-ac1f6b6435d0','c9806c74-bbca-11e8-b2bc-ac1f6b6435d0',
              '082a828a-bbbb-11e8-b2ba-ac1f6b6435d0'
             ]

for s in rods_rings:
    print ('{} -> {}'.format(data_set[data_set[:,0] == s], lookup[lookup[:,0] == s][:,1]))

[['e403806e-bbbf-11e8-b2bb-ac1f6b6435d0' '0']] -> ['27']
[['05d32f36-bba3-11e8-b2b9-ac1f6b6435d0' '1']] -> ['2 0 27']
[['0afda11a-bba0-11e8-b2b9-ac1f6b6435d0' '0']] -> ['19 4 27']
[['18df69fc-bbb5-11e8-b2ba-ac1f6b6435d0' '2']] -> ['5 0 27']
[['2b3ce424-bba8-11e8-b2ba-ac1f6b6435d0' '2']] -> ['5 0 27']
[['43f6bd88-bbc5-11e8-b2bc-ac1f6b6435d0' '2']] -> ['0 27']
[['70b97ed2-bbac-11e8-b2ba-ac1f6b6435d0' '2']] -> ['0 27']
[['802998d4-bbbb-11e8-b2ba-ac1f6b6435d0' '5']] -> ['23 27']
[['b1131086-bb9f-11e8-b2b9-ac1f6b6435d0' '5']] -> ['23 27']
[['c9806c74-bbca-11e8-b2bc-ac1f6b6435d0' '2']] -> ['5 0 27']
[['082a828a-bbbb-11e8-b2ba-ac1f6b6435d0' '2']] -> ['5 0 27']


In [14]:
cluster_sets = []
for i in range(n_clusters):
    cluster_sets.append(data_set[np.where(data_set[:,1]==str(i))])
cluster_sets = np.asarray(cluster_sets)


for i in range(n_clusters):
    print(cluster_sets[i].shape)


(7656, 2)
(2580, 2)
(7848, 2)
(4374, 2)
(1307, 2)
(2002, 2)
(1674, 2)
(3631, 2)


In [15]:
# Look at cluster contents
lookup = df.values
sample = []
for j in range(n_clusters):
    print('\n')
    print ('Cluster {} Examples'.format(j))
    for i in range(256):
        specimen_id = (random.choice(cluster_sets[j]))[0]
        print('Specimen: {} -> {}'.format(specimen_id, lookup[lookup[:,0] == specimen_id][:,1]))



Cluster 0 Examples
Specimen: 4c07eb1e-bba7-11e8-b2ba-ac1f6b6435d0 -> ['24 18']
Specimen: d2452d58-bbbc-11e8-b2ba-ac1f6b6435d0 -> ['4']
Specimen: 3a9abea2-bbc9-11e8-b2bc-ac1f6b6435d0 -> ['11']
Specimen: b07fa0e8-bb9b-11e8-b2b9-ac1f6b6435d0 -> ['6']
Specimen: 24b4817a-bba8-11e8-b2ba-ac1f6b6435d0 -> ['4']
Specimen: 69627f98-bbc6-11e8-b2bc-ac1f6b6435d0 -> ['19']
Specimen: c2f3ab48-bb9b-11e8-b2b9-ac1f6b6435d0 -> ['12 18']
Specimen: 61382bcc-bbaf-11e8-b2ba-ac1f6b6435d0 -> ['14']
Specimen: a7ec4b90-bbc6-11e8-b2bc-ac1f6b6435d0 -> ['3']
Specimen: 141f675a-bbc4-11e8-b2bc-ac1f6b6435d0 -> ['11']
Specimen: 084e28ee-bbab-11e8-b2ba-ac1f6b6435d0 -> ['19']
Specimen: bfe135e4-bb9e-11e8-b2b9-ac1f6b6435d0 -> ['6']
Specimen: bb432472-bbc9-11e8-b2bc-ac1f6b6435d0 -> ['21']
Specimen: 87882bac-bb9a-11e8-b2b9-ac1f6b6435d0 -> ['3']
Specimen: 97ec69ba-bbb6-11e8-b2ba-ac1f6b6435d0 -> ['14']
Specimen: 34a3289c-bbc7-11e8-b2bc-ac1f6b6435d0 -> ['14']
Specimen: cee7b06a-bbb0-11e8-b2ba-ac1f6b6435d0 -> ['19']
Specimen: 

Specimen: 39c89a8e-bbb6-11e8-b2ba-ac1f6b6435d0 -> ['9 10']
Specimen: bbe8c266-bbaf-11e8-b2ba-ac1f6b6435d0 -> ['19']
Specimen: c9042290-bbac-11e8-b2ba-ac1f6b6435d0 -> ['19']
Specimen: 413a1bd0-bba7-11e8-b2ba-ac1f6b6435d0 -> ['1']
Specimen: 6e70061c-bbae-11e8-b2ba-ac1f6b6435d0 -> ['21']
Specimen: 3d26dc4e-bbc6-11e8-b2bc-ac1f6b6435d0 -> ['4']
Specimen: 48a1f2c2-bba6-11e8-b2ba-ac1f6b6435d0 -> ['12']
Specimen: a686b280-bbb3-11e8-b2ba-ac1f6b6435d0 -> ['4']
Specimen: 0582608e-bbc0-11e8-b2bb-ac1f6b6435d0 -> ['6']
Specimen: be681478-bbba-11e8-b2ba-ac1f6b6435d0 -> ['21']
Specimen: bb665668-bbbf-11e8-b2bb-ac1f6b6435d0 -> ['21']
Specimen: e655b472-bba6-11e8-b2ba-ac1f6b6435d0 -> ['4']
Specimen: 90632af8-bba7-11e8-b2ba-ac1f6b6435d0 -> ['14']
Specimen: 0fe6b4b2-bbc0-11e8-b2bb-ac1f6b6435d0 -> ['4']
Specimen: b69186c6-bbba-11e8-b2ba-ac1f6b6435d0 -> ['18']
Specimen: 00ab10d6-bba4-11e8-b2b9-ac1f6b6435d0 -> ['22 6']
Specimen: 8437075e-bbc0-11e8-b2bb-ac1f6b6435d0 -> ['6']
Specimen: 284d75c6-bbbc-11e8-b2ba-

Specimen: fedb9b98-bbc3-11e8-b2bc-ac1f6b6435d0 -> ['2 0 21']
Specimen: bca81be4-bb9f-11e8-b2b9-ac1f6b6435d0 -> ['1 2']
Specimen: 4960cb9c-bbc0-11e8-b2bb-ac1f6b6435d0 -> ['2 0']
Specimen: 007811e6-bbbc-11e8-b2ba-ac1f6b6435d0 -> ['2']
Specimen: eb679172-bbad-11e8-b2ba-ac1f6b6435d0 -> ['2']
Specimen: 4b7646fe-bbc1-11e8-b2bb-ac1f6b6435d0 -> ['2 0']
Specimen: 8a44c8ec-bbb2-11e8-b2ba-ac1f6b6435d0 -> ['2 0']
Specimen: d6596e32-bbb1-11e8-b2ba-ac1f6b6435d0 -> ['2 0']
Specimen: ee439aea-bbc5-11e8-b2bc-ac1f6b6435d0 -> ['2']
Specimen: 57132c62-bbb1-11e8-b2ba-ac1f6b6435d0 -> ['2 0']
Specimen: 8fd17e5e-bbad-11e8-b2ba-ac1f6b6435d0 -> ['1 2 0']
Specimen: 405e062a-bbc7-11e8-b2bc-ac1f6b6435d0 -> ['2']
Specimen: adbd16fc-bbbd-11e8-b2ba-ac1f6b6435d0 -> ['24 2 0']
Specimen: c42ccc62-bbbc-11e8-b2ba-ac1f6b6435d0 -> ['2 0']
Specimen: e7c486fa-bb9a-11e8-b2b9-ac1f6b6435d0 -> ['2']
Specimen: 17147fb2-bbb1-11e8-b2ba-ac1f6b6435d0 -> ['2 0']
Specimen: 8bdaeab0-bb9e-11e8-b2b9-ac1f6b6435d0 -> ['6 2']
Specimen: 3b8734

Specimen: d753d14a-bbae-11e8-b2ba-ac1f6b6435d0 -> ['0 21']
Specimen: 5f37d732-bbaf-11e8-b2ba-ac1f6b6435d0 -> ['0']
Specimen: d01fb1a6-bb9e-11e8-b2b9-ac1f6b6435d0 -> ['16 0']
Specimen: 114dbe4c-bbc3-11e8-b2bc-ac1f6b6435d0 -> ['5 0']
Specimen: 4db30bbe-bbb2-11e8-b2ba-ac1f6b6435d0 -> ['7 0']
Specimen: 1694b480-bbb1-11e8-b2ba-ac1f6b6435d0 -> ['19 0']
Specimen: 07d94f6e-bbac-11e8-b2ba-ac1f6b6435d0 -> ['0 21']
Specimen: 4da64b48-bbc3-11e8-b2bc-ac1f6b6435d0 -> ['0']
Specimen: b2020902-bbbd-11e8-b2ba-ac1f6b6435d0 -> ['23 0']
Specimen: c5aef1fa-bbbc-11e8-b2ba-ac1f6b6435d0 -> ['0']
Specimen: 9068890c-bbb2-11e8-b2ba-ac1f6b6435d0 -> ['5 0']
Specimen: 265a8e88-bbb8-11e8-b2ba-ac1f6b6435d0 -> ['0']
Specimen: db698ad8-bba2-11e8-b2b9-ac1f6b6435d0 -> ['0']
Specimen: 5e605e06-bbb4-11e8-b2ba-ac1f6b6435d0 -> ['5 0']
Specimen: d9087220-bbc7-11e8-b2bc-ac1f6b6435d0 -> ['0']
Specimen: 7c63dda8-bbb2-11e8-b2ba-ac1f6b6435d0 -> ['0']
Specimen: 1bb9c98e-bbb9-11e8-b2ba-ac1f6b6435d0 -> ['0']
Specimen: 5af9d8d8-bba9-1

Specimen: 504c80e0-bb9d-11e8-b2b9-ac1f6b6435d0 -> ['23 0']
Specimen: 2632b0cc-bba2-11e8-b2b9-ac1f6b6435d0 -> ['0']
Specimen: 96cac5c8-bbc4-11e8-b2bc-ac1f6b6435d0 -> ['7 0']
Specimen: 2cea24d4-bba4-11e8-b2b9-ac1f6b6435d0 -> ['23 3 0']
Specimen: 23738388-bbbc-11e8-b2ba-ac1f6b6435d0 -> ['0']
Specimen: 08cd1a98-bbb8-11e8-b2ba-ac1f6b6435d0 -> ['11 5 0']
Specimen: f6823a8e-bba9-11e8-b2ba-ac1f6b6435d0 -> ['3 0']
Specimen: ab3b9258-bbab-11e8-b2ba-ac1f6b6435d0 -> ['0 21']
Specimen: c481dfd2-bbc5-11e8-b2bc-ac1f6b6435d0 -> ['19 0']
Specimen: bfbcc8b4-bbb0-11e8-b2ba-ac1f6b6435d0 -> ['0 21']
Specimen: 8ad0a8ee-bbc6-11e8-b2bc-ac1f6b6435d0 -> ['19 5 0']
Specimen: bad352ca-bba9-11e8-b2ba-ac1f6b6435d0 -> ['0 21']
Specimen: 45b44cf6-bbb8-11e8-b2ba-ac1f6b6435d0 -> ['19 0 21']
Specimen: a33c95ee-bba0-11e8-b2b9-ac1f6b6435d0 -> ['1 0']
Specimen: 8d066aca-bba5-11e8-b2ba-ac1f6b6435d0 -> ['1 0 21']
Specimen: 7b4c30fe-bbae-11e8-b2ba-ac1f6b6435d0 -> ['0']
Specimen: 46277d7a-bbbd-11e8-b2ba-ac1f6b6435d0 -> ['0 21'

...

Specimen: 4137e740-bba5-11e8-b2ba-ac1f6b6435d0 -> ['23 4']
Specimen: 976e8f20-bbc7-11e8-b2bc-ac1f6b6435d0 -> ['23']
Specimen: 80835882-bbb2-11e8-b2ba-ac1f6b6435d0 -> ['23']
Specimen: 634d50de-bbc5-11e8-b2bc-ac1f6b6435d0 -> ['23']
Specimen: 705103f2-bbbb-11e8-b2ba-ac1f6b6435d0 -> ['23']
Specimen: e5374b3a-bba3-11e8-b2b9-ac1f6b6435d0 -> ['23']
Specimen: 836a3fb4-bba5-11e8-b2ba-ac1f6b6435d0 -> ['23']
Specimen: 334aa47a-bba4-11e8-b2b9-ac1f6b6435d0 -> ['23']
Specimen: 18618f3a-bba3-11e8-b2b9-ac1f6b6435d0 -> ['23']
Specimen: 6b491864-bbc4-11e8-b2bc-ac1f6b6435d0 -> ['23']
Specimen: 9daafc3c-bbbf-11e8-b2bb-ac1f6b6435d0 -> ['23']
Specimen: 1ee81466-bbc9-11e8-b2bc-ac1f6b6435d0 -> ['23']
Specimen: f79044c2-bb9d-11e8-b2b9-ac1f6b6435d0 -> ['23']
Specimen: a9c73924-bbbd-11e8-b2ba-ac1f6b6435d0 -> ['23']
Specimen: d99a7f80-bbbd-11e8-b2ba-ac1f6b6435d0 -> ['23']
Specimen: 4c0c2348-bb9b-11e8-b2b9-ac1f6b6435d0 -> ['19 23']
Specimen: bdf2df6e-bbb0-11e8-b2ba-ac1f6b6435d0 -> ['23']
Specimen: 2105213a-bba1-11

Specimen: 60a32648-bbba-11e8-b2ba-ac1f6b6435d0 -> ['23']
Specimen: 84cf07f2-bbbb-11e8-b2ba-ac1f6b6435d0 -> ['23']


Cluster 6 Examples
Specimen: 76527ba6-bbc4-11e8-b2bc-ac1f6b6435d0 -> ['7']
Specimen: cd4b869c-bbb4-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: ba675ebe-bbb1-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: bdaf2d02-bb9a-11e8-b2b9-ac1f6b6435d0 -> ['7']
Specimen: 69dacdb2-bb9a-11e8-b2b9-ac1f6b6435d0 -> ['7']
Specimen: 2d974fe0-bbab-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: fb591ef2-bba3-11e8-b2b9-ac1f6b6435d0 -> ['25 7']
Specimen: 3bd42398-bbac-11e8-b2ba-ac1f6b6435d0 -> ['7 2']
Specimen: a28bc7e6-bbbe-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: e8fab6b4-bb9c-11e8-b2b9-ac1f6b6435d0 -> ['12 25 7']
Specimen: a17a6238-bba8-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: 009b8c20-bbc6-11e8-b2bc-ac1f6b6435d0 -> ['7']
Specimen: 5d3b8e2a-bbb3-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: 90ccada4-bbc8-11e8-b2bc-ac1f6b6435d0 -> ['7']
Specimen: 6a05e118-bb9b-11e8-b2b9-ac1f6b6435d0 -> ['7']
Specimen: 2f46

Specimen: e955f0a8-bbc7-11e8-b2bc-ac1f6b6435d0 -> ['7 18']
Specimen: 30747832-bba6-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: c3a50ec2-bba2-11e8-b2b9-ac1f6b6435d0 -> ['7']
Specimen: f41c602e-bbc4-11e8-b2bc-ac1f6b6435d0 -> ['7']
Specimen: f80b5238-bba8-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: 7bfee190-bba9-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: 58b0c540-bba0-11e8-b2b9-ac1f6b6435d0 -> ['7']
Specimen: d5358e5e-bb9e-11e8-b2b9-ac1f6b6435d0 -> ['7']
Specimen: 5e979a20-bbbd-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: bcb03e46-bbb8-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: 5028afd2-bbc8-11e8-b2bc-ac1f6b6435d0 -> ['7']
Specimen: d67119fe-bbae-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: 954ea85c-bba8-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: ac83064c-bba5-11e8-b2ba-ac1f6b6435d0 -> ['7']
Specimen: 65a6e836-bbc5-11e8-b2bc-ac1f6b6435d0 -> ['7']
Specimen: f7596a70-bbab-11e8-b2ba-ac1f6b6435d0 -> ['25 7']
Specimen: 34c4aca8-bbc5-11e8-b2bc-ac1f6b6435d0 -> ['7 2']
Specimen: 65653ac4-bbc7-11e8-b2bc-ac1f6b

Specimen: 26a0e4f6-bbc6-11e8-b2bc-ac1f6b6435d0 -> ['25 2 0']
Specimen: 5b8ab080-bba7-11e8-b2ba-ac1f6b6435d0 -> ['25 0 21']
Specimen: 61a3c1fc-bbaf-11e8-b2ba-ac1f6b6435d0 -> ['25 0 21']
Specimen: 6c1b761c-bba9-11e8-b2ba-ac1f6b6435d0 -> ['25 7 0']
Specimen: 4629c9be-bbc1-11e8-b2bb-ac1f6b6435d0 -> ['25 0']
Specimen: 2bf66106-bbb7-11e8-b2ba-ac1f6b6435d0 -> ['25 0 21']
Specimen: 68e4bb78-bbc9-11e8-b2bc-ac1f6b6435d0 -> ['16 25 0']
Specimen: 12cac722-bbca-11e8-b2bc-ac1f6b6435d0 -> ['25 0']
Specimen: da78e1d4-bbc3-11e8-b2bc-ac1f6b6435d0 -> ['25 7 0']
Specimen: 49a445fa-bbc7-11e8-b2bc-ac1f6b6435d0 -> ['25 2 0']
Specimen: 1b3cf15c-bbb9-11e8-b2ba-ac1f6b6435d0 -> ['25 0']
Specimen: 19405cae-bbb4-11e8-b2ba-ac1f6b6435d0 -> ['25 0']
Specimen: c5a5a934-bbca-11e8-b2bc-ac1f6b6435d0 -> ['25 0']
Specimen: ffaebd74-bb9c-11e8-b2b9-ac1f6b6435d0 -> ['25 5 0']
Specimen: a3e41156-bbc4-11e8-b2bc-ac1f6b6435d0 -> ['25 0 21']
Specimen: 6990d6c0-bba5-11e8-b2ba-ac1f6b6435d0 -> ['19 25 0']
Specimen: a6ae47a0-bbc7-11e8

Specimen: d5f382b6-bbca-11e8-b2bc-ac1f6b6435d0 -> ['25 0']
Specimen: 52c95216-bb9e-11e8-b2b9-ac1f6b6435d0 -> ['25 0']
Specimen: 2a1d43c0-bbbe-11e8-b2ba-ac1f6b6435d0 -> ['25 0']
Specimen: 14f2225a-bbbd-11e8-b2ba-ac1f6b6435d0 -> ['25 0']
Specimen: 5e638e90-bbc9-11e8-b2bc-ac1f6b6435d0 -> ['25 7 0']


In [16]:
# get kfold 0
val_set = []
val_set_already_used = []
for i in range(n_clusters):
    for cluster_set in cluster_sets[i]:
        if np.random.uniform() > 0.9:
            val_set.append(cluster_set[0])

for v in val_set:
    val_set_already_used.append(v)
    
print('Kfold Length -> {}'.format(len(val_set)))
print('Used  Length -> {}'.format(len(val_set_already_used)))
df_val_set = pd.DataFrame(np.asarray(val_set))
df_val_set.to_csv('../val_set_kfold_0.csv', index=False, header=['Id'])
df_val_set.head(5)

            

Kfold Length -> 3075
Used  Length -> 3075


,0
0,0047c984-bba6-11e8-b2ba-ac1f6b6435d0
1,004bf4c6-bbc6-11e8-b2bc-ac1f6b6435d0
2,006f5b04-bbb3-11e8-b2ba-ac1f6b6435d0
3,0216d526-bbbe-11e8-b2ba-ac1f6b6435d0
4,02e2a5d6-bbad-11e8-b2ba-ac1f6b6435d0


In [17]:
# get kfold 1
val_set = []
for i in range(n_clusters):
    for cluster_set in cluster_sets[i]:
        if cluster_set[0] in val_set_already_used:
            pass
        elif np.random.uniform() > 0.89:
            val_set.append(cluster_set[0])

for v in val_set:
    val_set_already_used.append(v)
    
print('Kfold Length -> {}'.format(len(val_set)))
print('Used  Length -> {}'.format(len(val_set_already_used)))
df_val_set = pd.DataFrame(np.asarray(val_set))
df_val_set.to_csv('../val_set_kfold_1.csv', index=False, header=['Id'])
df_val_set.head(5)


Kfold Length -> 3081
Used  Length -> 6156


,0
0,00638dd0-bbac-11e8-b2ba-ac1f6b6435d0
1,0088a1b6-bbb5-11e8-b2ba-ac1f6b6435d0
2,0092d59c-bba5-11e8-b2ba-ac1f6b6435d0
3,0101c0be-bba6-11e8-b2ba-ac1f6b6435d0
4,017f7748-bba6-11e8-b2ba-ac1f6b6435d0


In [18]:
# get kfold 2
val_set = []
for i in range(n_clusters):
    for cluster_set in cluster_sets[i]:
        if cluster_set[0] in val_set_already_used:
            pass
        elif np.random.uniform() > 0.875:
            val_set.append(cluster_set[0])

for v in val_set:
    val_set_already_used.append(v)

print('Kfold Length -> {}'.format(len(val_set)))
print('Used  Length -> {}'.format(len(val_set_already_used)))
df_val_set = pd.DataFrame(np.asarray(val_set))
df_val_set.to_csv('../val_set_kfold_2.csv', index=False, header=['Id'])
df_val_set.head(5)


Kfold Length -> 3110
Used  Length -> 9266


,0
0,008761b4-bbad-11e8-b2ba-ac1f6b6435d0
1,00b9bdcc-bbca-11e8-b2bc-ac1f6b6435d0
2,00ed4394-bbb2-11e8-b2ba-ac1f6b6435d0
3,01106f56-bbc9-11e8-b2bc-ac1f6b6435d0
4,0136e24a-bba0-11e8-b2b9-ac1f6b6435d0


In [19]:
# get kfold 3
val_set = []
for i in range(n_clusters):
    for cluster_set in cluster_sets[i]:
        if cluster_set[0] in val_set_already_used:
            pass
        elif np.random.uniform() > 0.86:
            val_set.append(cluster_set[0])

for v in val_set:
    val_set_already_used.append(v)

print('Kfold Length -> {}'.format(len(val_set)))
print('Used  Length -> {}'.format(len(val_set_already_used)))
df_val_set = pd.DataFrame(np.asarray(val_set))
df_val_set.to_csv('../val_set_kfold_3.csv', index=False, header=['Id'])
df_val_set.head(5)


Kfold Length -> 3078
Used  Length -> 12344


,0
0,00301238-bbb2-11e8-b2ba-ac1f6b6435d0
1,00df1310-bbca-11e8-b2bc-ac1f6b6435d0
2,01747a40-bbc0-11e8-b2bb-ac1f6b6435d0
3,017badb4-bbc6-11e8-b2bc-ac1f6b6435d0
4,01f3e0da-bb9f-11e8-b2b9-ac1f6b6435d0


In [20]:
# get kfold 4
val_set = []
for i in range(n_clusters):
    for cluster_set in cluster_sets[i]:
        if cluster_set[0] in val_set_already_used:
            pass
        elif np.random.uniform() > 0.835:
            val_set.append(cluster_set[0])

for v in val_set:
    val_set_already_used.append(v)

print('Kfold Length -> {}'.format(len(val_set)))
print('Used  Length -> {}'.format(len(val_set_already_used)))
df_val_set = pd.DataFrame(np.asarray(val_set))
df_val_set.to_csv('../val_set_kfold_4.csv', index=False, header=['Id'])
df_val_set.head(5)


Kfold Length -> 3180
Used  Length -> 15524


,0
0,001838f8-bbca-11e8-b2bc-ac1f6b6435d0
1,006cde8a-bbbb-11e8-b2ba-ac1f6b6435d0
2,00cb9586-bbc2-11e8-b2bb-ac1f6b6435d0
3,00e77c9c-bbc2-11e8-b2bb-ac1f6b6435d0
4,01028112-bbb4-11e8-b2ba-ac1f6b6435d0
